# Weight Forecasting Model Development

## Setup

In [18]:
import sys
toolpath = '/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/src'
sys.path.append(toolpath)

%load_ext autoreload
%autoreload 2

from wtfc_utils import etl_utils as etl

from sqlalchemy import create_engine
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [19]:
server_dir = '/Users/jamieinfinity/Dropbox/Projects/WeightForecaster/weightforecaster/server/'
db_dir = server_dir + 'db/'
db_name = 'weightforecaster'
db_ext = '.db'
db_file_name = db_dir + db_name + db_ext

In [20]:
# See: https://pandas.pydata.org/pandas-docs/stable/io.html#advanced-sqlalchemy-queries
engine = create_engine('sqlite:///'+db_file_name)

In [21]:
with engine.connect() as conn, conn.begin():
    data_df = pd.read_sql_table('fitness', conn, index_col='date', parse_dates=['date'])
    
temp_df = pd.DataFrame(index=pd.date_range(start="2015-09-16",end=data_df.index.max()))
data_df = pd.merge(temp_df, data_df, how='left', left_index=True, right_index=True)
data_df = data_df[data_df.w_7day_avg_weekly_diff>-3].copy() # drop outliers due to imputation effects
data_df.dropna(inplace=True)
data_df['date'] = data_df.index
data_df.rename({
    'w_7day_avg':'w',
    'c_7day_avg':'c',
    's_7day_avg':'s',
    'w_7day_avg_last_week':'w_prev',
    'c_7day_avg_last_week':'c_prev',
    's_7day_avg_last_week':'s_prev',
    'w_7day_avg_weekly_diff':'dw',
}, axis=1, inplace=True)

In [22]:
data_df.tail(5)

,weight,calories,steps,weight_imputed,w,c,s,w_prev,c_prev,s_prev,dw,date
2021-01-21,152.9,2192.0,16856.0,0.0,153.857143,1913.000000,16166.714286,155.771429,2300.285714,12039.571429,-1.914286,2021-01-21
2021-01-22,152.9,2005.0,13643.0,0.0,153.257143,1878.285714,16550.571429,155.828571,2334.428571,11951.000000,-2.571429,2021-01-22
2021-01-24,152.7,2130.0,11859.0,0.0,152.728571,2017.428571,15560.714286,155.714286,2198.571429,13487.714286,-2.985714,2021-01-24
2021-01-25,152.4,2265.0,11149.0,0.0,152.814286,2053.571429,15047.285714,155.257143,2167.714286,14029.714286,-2.442857,2021-01-25
2021-01-26,153.8,1907.0,10308.0,0.0,152.985714,2088.000000,13471.000000,154.800000,2065.714286,14642.142857,-1.814286,2021-01-26


## Linear model development

In [23]:
alt.Chart(data_df).mark_circle(size=60).encode(
    x=alt.X('c',
        scale=alt.Scale(domain=(1400, 3200))
    ), 
    y=alt.X('dw',
        scale=alt.Scale(domain=(-3, 3))
    ),    
    color='s',
    tooltip=['date', 'w', 'c', 's']
).properties(
    width=500,
    height=300
).interactive()

alt.Chart(...)

### Train/test split

In [24]:
data_modeling = data_df[['w_prev','c_prev','s_prev','c','s','w']].copy()
data_train = data_modeling[data_modeling.index<'2019-11-01'].copy()
data_test = data_modeling[(data_modeling.index>='2019-12-01')&(data_modeling.index<='2020-04-22')].copy()
data_test['date'] = data_test.index

features=['w_prev','c','s']
x_train = data_train[features].values
y_train = data_train[['w']].values
x_test = data_test[features].values
y_test = data_test[['w']].values

### Train the model

In [25]:
model = LinearRegression().fit(x_train, y_train)

In [26]:
model.score(x_train, y_train)

0.9846420709902819

In [27]:
model.score(x_test, y_test)

0.9802069429646699

### Model performance on test set

In [29]:
y_pred = model.predict(x_test)
data_test['w_pred'] = [x[0] for x in y_pred]

In [30]:
ys = (165, 181)
w=alt.Chart(data_test).mark_circle(size=60, opacity=0.4, color='black').encode(
    x='date',   
    y=alt.Y('w',
        scale=alt.Scale(domain=ys)
    ),      
    tooltip=['date', 'w', 'c', 's']
).properties(
    width=500,
    height=300
).interactive()
wp=alt.Chart(data_test).mark_point(size=60,opacity=0.5, color='red').encode(
    x='date',   
    y=alt.Y('w_pred',
        scale=alt.Scale(domain=ys)
    ),      
    tooltip=['date', 'w', 'c', 's']
).interactive()
wp+w

alt.LayerChart(...)

### Weekly weight change vs calories

In [31]:
[c_w, c_c, c_s] = list(model.coef_[0])
c_0 = model.intercept_[0]

weight_prev = [160, 170, 180]
steps = [5000, 10000, 15000]
cals = list(range(1500, 3000, 10))

wgt = []
for wp in weight_prev:
    for s in steps:
        wgt_df = pd.DataFrame({'calories':cals})
        w = c_0 + c_w*wp + c_c*np.array(cals) + c_s*s
        wgt_df['delta_w'] = w-wp
        wgt_df['weight'] = wp
        wgt_df['steps'] = s
        wgt.append(wgt_df)
wgt_df = pd.concat(wgt, ignore_index=True)

In [130]:
wgt = wgt_df[wgt_df.steps==10000].copy()
alt.Chart(wgt).mark_circle(size=60).encode(
    alt.X('calories',
        scale=alt.Scale(domain=(1600, 2800))
    ),    
    y='delta_w',
    color='weight',
    tooltip=['calories', 'steps', 'weight', 'delta_w']
).properties(
    width=500,
    height=300
).interactive()

alt.Chart(...)

In [131]:
wgt = wgt_df[wgt_df.weight==170].copy()
alt.Chart(wgt).mark_circle(size=60).encode(
    alt.X('calories',
        scale=alt.Scale(domain=(1600, 2800))
    ),    
    y='delta_w',
    color='steps',
    tooltip=['calories', 'steps', 'weight', 'delta_w']
).properties(
    width=500,
    height=300
).interactive()

alt.Chart(...)

### Forecasting with actual (c,s) trajectory

In [132]:
wgt_forecast = data_test[['date','w_prev','w','c','s']].copy()
wgt_forecast['w_fc'] = wgt_forecast['w']

for dt in list(wgt_forecast.index)[7:]:
    df = wgt_forecast.loc[dt]    
    dt_last_week = dt +  + datetime.timedelta(days=-7)
    df_last_week = wgt_forecast.loc[dt_last_week]
    w_prev = df_last_week.w_fc
#     w_prev = df.w_prev
    c = df.c
    s = df.s
#     w_fc = c_0 + c_w*w_prev + c_c*c + c_s*s
    w_fc = model.predict([[w_prev, c, s]])
    wgt_forecast.loc[wgt_forecast.date==dt, 'w_fc'] = w_fc


# wgt_forecast = data_test[['date','w_prev','w','c','s']].copy()

# wprev = list(wgt_forecast.w_prev.iloc[0:7].values)
# w_fc = []

# index=0
# for dt in wgt_forecast.index:
#     df = wgt_forecast.loc[dt]
#     w = c_0 + c_w*wprev[index] + c_c*df.c + c_s*df.s
#     w_fc.append(w)
#     wprev.append(w)
#     index+=1
# wgt_forecast['w_fc'] = w_fc
# wgt_forecast['wprev_fc'] = wprev[7:]

In [133]:
ch1=alt.Chart(wgt_forecast).mark_circle(size=60).encode(
    x='date',  
    y=alt.Y('w',
        scale=alt.Scale(domain=(165, 180))
    ),  
    tooltip=['date', 'w', 'w_fc', 'c', 's', 'w_prev']
).properties(
    width=500,
    height=300
).interactive()
ch2=alt.Chart(wgt_forecast).mark_circle(size=60,color='red', opacity=0.3).encode(
    x='date',  
    y=alt.Y('w_fc',
        scale=alt.Scale(domain=(170, 180))
    ),  
    tooltip=['date', 'w', 'w_fc', 'c', 's', 'w_prev']
).properties(
    width=500,
    height=300
).interactive()
ch1+ch2

alt.LayerChart(...)

## Final model for forecasting

### Train/test split

In [170]:
# note: this is for the 'final' model, we use all of the data to train
# note: we still break off a chunk for seeing how final model performs 

data_modeling = data_df[['w_prev','c_prev','s_prev','c','s','w']].copy()
data_train = data_modeling[data_modeling.index<datetime.datetime.today()].copy()
data_test = data_modeling[data_modeling.index>='2020-01-01'].copy()
data_test['date'] = data_test.index

features=['w_prev','c','s']
x_train = data_train[features].values
y_train = data_train[['w']].values
x_test = data_test[features].values
y_test = data_test[['w']].values

### Train the model

In [171]:
model = LinearRegression().fit(x_train, y_train)

In [172]:
model.score(x_train, y_train)

0.9833767261807124

In [173]:
model.score(x_test, y_test)

0.9728349932255428

### Model performance on test set

In [174]:
y_pred = model.predict(x_test)
data_test['w_pred'] = [x[0] for x in y_pred]

In [175]:
ys = (165, 181)
w=alt.Chart(data_test).mark_circle(size=60, opacity=0.4, color='black').encode(
    x='date',   
    y=alt.Y('w',
        scale=alt.Scale(domain=ys)
    ),      
    tooltip=['date', 'w', 'c', 's']
).properties(
    width=500,
    height=300
).interactive()
wp=alt.Chart(data_test).mark_point(size=60,opacity=0.5, color='red').encode(
    x='date',   
    y=alt.Y('w_pred',
        scale=alt.Scale(domain=ys)
    ),      
    tooltip=['date', 'w', 'c', 's']
).interactive()
wp+w

alt.LayerChart(...)

### Weekly weight change vs calories

In [176]:
[c_w, c_c, c_s] = list(model.coef_[0])
c_0 = model.intercept_[0]

weight_prev = [160, 170, 180]
steps = [5000, 10000, 15000]
cals = list(range(1500, 3000, 10))

wgt = []
for wp in weight_prev:
    for s in steps:
        wgt_df = pd.DataFrame({'calories':cals})
        w = c_0 + c_w*wp + c_c*np.array(cals) + c_s*s
        wgt_df['delta_w'] = w-wp
        wgt_df['weight'] = wp
        wgt_df['steps'] = s
        wgt.append(wgt_df)
wgt_df = pd.concat(wgt, ignore_index=True)

In [177]:
wgt = wgt_df[wgt_df.steps==10000].copy()
alt.Chart(wgt).mark_circle(size=60).encode(
    alt.X('calories',
        scale=alt.Scale(domain=(1600, 2800))
    ),    
    y='delta_w',
    color='weight',
    tooltip=['calories', 'steps', 'weight', 'delta_w']
).properties(
    width=500,
    height=300
).interactive()

alt.Chart(...)

In [178]:
wgt = wgt_df[wgt_df.weight==170].copy()
alt.Chart(wgt).mark_circle(size=60).encode(
    alt.X('calories',
        scale=alt.Scale(domain=(1600, 2800))
    ),    
    y='delta_w',
    color='steps',
    tooltip=['calories', 'steps', 'weight', 'delta_w']
).properties(
    width=500,
    height=300
).interactive()

alt.Chart(...)

### Forecasting with actual (c,s) trajectory

In [179]:
wgt_forecast = data_test[['date','w_prev','w','c','s']].copy()
wgt_forecast['w_fc'] = wgt_forecast['w']

for dt in list(wgt_forecast.index)[7:]:
    df = wgt_forecast.loc[dt]    
    dt_last_week = dt +  + datetime.timedelta(days=-7)
    df_last_week = wgt_forecast.loc[dt_last_week]
    w_prev = df_last_week.w_fc
#     w_prev = df.w_prev
    c = df.c
    s = df.s
    w_fc = c_0 + c_w*w_prev + c_c*c + c_s*s
#     w_fc = model.predict([[w_prev, c, s]])
    wgt_forecast.loc[wgt_forecast.date==dt, 'w_fc'] = w_fc

In [180]:
ch1=alt.Chart(wgt_forecast).mark_circle(size=60).encode(
    x='date',  
    y=alt.Y('w',
        scale=alt.Scale(domain=(165, 180))
    ),  
    tooltip=['date', 'w', 'w_fc', 'c', 's', 'w_prev']
).properties(
    width=500,
    height=300
).interactive()
ch2=alt.Chart(wgt_forecast).mark_circle(size=60,color='red', opacity=0.3).encode(
    x='date',  
    y=alt.Y('w_fc',
        scale=alt.Scale(domain=(170, 180))
    ),  
    tooltip=['date', 'w', 'w_fc', 'c', 's', 'w_prev']
).properties(
    width=500,
    height=300
).interactive()
ch1+ch2

alt.LayerChart(...)

### Forecasting function

NOTES:

- always fix target weight
- scenario 1: fix steps, calories, solve for date/days
- scenario 2: fix steps, date/days, solve for calories

Future:
- scenario 3: fix calories, date/days, solve for steps
- scenario 4: instead of fixing target weight, fix lbs/week, then repeat scenarios 1 to 3

In [249]:
def weight_forecast_i(model_coefs, w_prev, c_i, s_i):
    [c_w, c_c, c_s, c_0] = model_coefs
    return c_0 + c_w*w_prev + c_c*c_i + c_s*s_i

def weight_forecast(model_coefs, w_prev_init, calories, steps, num_weeks, date_init):
    d_init = date_init
    if type(date_init)==str:
        d_init = datetime.datetime.strptime(date_init, '%Y-%m-%d').date()
    d = date_init
    w = w_prev_init
    weeks = list(range(1, num_weeks+1))
    dates = [d]    
    wgt = [w]
    for week in weeks:
        d = d + datetime.timedelta(days=7)
        w = weight_forecast_i(model_coefs, w, calories, steps)
        dates.append(d)
        wgt.append(w)
    weeks = [0] + weeks
    return [weeks, dates, wgt]

In [613]:
[c_w, c_c, c_s, c_0] = list(model.coef_[0]) + [model.intercept_[0]]
model_coefs = [c_w, c_c, c_s, c_0]
model_coefs

[0.9842664081035283,
 0.001965638199353011,
 -4.621900527451458e-05,
 -1.2110620297640367]

In [277]:
# [weeks, dates, wgt] = weight_forecast(model_coefs, 169.5, 1800, 10000, 100, '2020-04-23')
[weeks, dates, wgt] = weight_forecast(model_coefs, 169.5, 1800, 3000, 100, datetime.date.today())

In [278]:
[(weeks[i], dates[i], wgt[i]) for i in range(len(wgt))]

[(0, datetime.date(2020, 4, 23), 169.5),
 (1, datetime.date(2020, 4, 30), 169.0215858867959),
 (2, datetime.date(2020, 5, 7), 168.55069894600643),
 (3, datetime.date(2020, 5, 14), 168.08722074817274),
 (4, datetime.date(2020, 5, 21), 167.63103472715667),
 (5, datetime.date(2020, 5, 28), 167.18202615082416),
 (6, datetime.date(2020, 6, 4), 166.74008209218968),
 (7, datetime.date(2020, 6, 11), 166.3050914010148),
 (8, datetime.date(2020, 6, 18), 165.87694467585365),
 (9, datetime.date(2020, 6, 25), 165.455534236538),
 (10, datetime.date(2020, 7, 2), 165.04075409709546),
 (11, datetime.date(2020, 7, 9), 164.63249993909366),
 (12, datetime.date(2020, 7, 16), 164.2306690854039),
 (13, datetime.date(2020, 7, 23), 163.8351604743775),
 (14, datetime.date(2020, 7, 30), 163.44587463442852),
 (15, datetime.date(2020, 8, 6), 163.0627136590164),
 (16, datetime.date(2020, 8, 13), 162.68558118202205),
 (17, datetime.date(2020, 8, 20), 162.31438235351163),
 (18, datetime.date(2020, 8, 27), 161.9490238

In [522]:
def get_forecast_data(model_coefs, wgt_init=None, cals=None, steps=None, max_num_weeks=26, dw=1, dc=100, ds=1000):
    date_init = datetime.date.today()
    wgt_init_list = [wgt_init]
    cals_list = [cals]
    steps_list = [steps]
    if wgt_init==None:
        wgt_init_list = np.arange(149, 181, dw)
    if cals==None:
        cals_list = np.arange(1000, 2600, dc)
    if steps==None:
        steps_list = np.arange(1000, 20000, ds)

    forecast_data = []
    for wgt_init in wgt_init_list:
        for cal in cals_list:
            for steps in steps_list:
                [weeks, dates, wgt] = weight_forecast(model_coefs, wgt_init, cal, steps, max_num_weeks, date_init)
                fc_df = pd.DataFrame({'week':weeks, 'date':dates, 'weight':wgt})
                fc_df['weight_init'] = wgt_init
                fc_df['calories'] = cal
                fc_df['steps'] = steps
                forecast_data.append(fc_df)
    forecast_data = pd.concat(forecast_data, ignore_index=True)
    return forecast_data

In [523]:
foo = get_forecast_data(model_coefs, wgt_init=169.5, cals=1800, steps=3000, max_num_weeks=52)
print(foo.shape)
foo

(53, 6)


,week,date,weight,weight_init,calories,steps
0,0,2020-04-26,169.500000,169.5,1800,3000
1,1,2020-05-03,169.021586,169.5,1800,3000
2,2,2020-05-10,168.550699,169.5,1800,3000
3,3,2020-05-17,168.087221,169.5,1800,3000
4,4,2020-05-24,167.631035,169.5,1800,3000
5,5,2020-05-31,167.182026,169.5,1800,3000
6,6,2020-06-07,166.740082,169.5,1800,3000
7,7,2020-06-14,166.305091,169.5,1800,3000
8,8,2020-06-21,165.876945,169.5,1800,3000
9,9,2020-06-28,165.455534,169.5,1800,3000


In [537]:
foo = get_forecast_data(model_coefs, wgt_init=169.5, steps=3000, max_num_weeks=52, dc=20)
print(foo.shape)
foo

(4240, 6)


,week,date,weight,weight_init,calories,steps
0,0,2020-04-27,169.500000,169.5,1000,3000
1,1,2020-05-04,167.449075,169.5,1000,3000
2,2,2020-05-11,165.430419,169.5,1000,3000
3,3,2020-05-18,163.443524,169.5,1000,3000
4,4,2020-05-25,161.487889,169.5,1000,3000
...,...,...,...,...,...,...
4235,48,2021-03-29,205.225887,169.5,2580,3000
4236,49,2021-04-05,205.718574,169.5,2580,3000
4237,50,2021-04-12,206.203509,169.5,2580,3000
4238,51,2021-04-19,206.680815,169.5,2580,3000


In [609]:
def solve_for_weeks(model_coefs, weight_init, weight_target, steps_target, calories_target):
    forecast_data = get_forecast_data(model_coefs, wgt_init=weight_init, 
                                      cals=calories_target, steps=steps_target, max_num_weeks=52)
    forecast_data.sort_values('date', inplace=True)
    forecast_data['weight_prev'] = forecast_data.weight.shift(1)
    forecast_data['dw'] = forecast_data.weight - forecast_data.weight_prev
    if min(forecast_data.weight>weight_target):
        dw = np.round(forecast_data.dw.mean(),2)
        return [None, None, dw]
    forecast_data = forecast_data[forecast_data.weight>(weight_target-2)].copy()
    dw = np.round(forecast_data.dw.mean(),1)   
    interp = interp1d(forecast_data.weight, forecast_data.week)
    weeks = interp(weight_target)
    date = datetime.datetime.today() + datetime.timedelta(days=(round(weeks*7)))
    weeks = np.round(weeks, 2)
    return [weeks, date.date(), dw, steps_target, calories_target]

def solve_for_calories(model_coefs, weight_init, weight_target, steps_target, date_target):
    d_target = date_target
    if type(d_target)==str:
        d_target = datetime.datetime.strptime(d_target, '%Y-%m-%d').date()
    w_target = np.round((d_target - datetime.date.today()).days/7.0,1)
    forecast_data = get_forecast_data(model_coefs, wgt_init=weight_init, steps=steps_target, dc=1, max_num_weeks=52)
    forecast_data = forecast_data[
        (forecast_data.week<(w_target+1)) 
        & (forecast_data.week>(w_target-1))
        & (forecast_data.weight<(weight_target+1))
        & (forecast_data.weight>(weight_target-1))].copy()
    candidate_cal = list(forecast_data.calories.unique())
    candidate_date = [solve_for_weeks(model_coefs, weight_init, weight_target, steps_target, c)[1] for c in candidate_cal]
    temp_df = pd.DataFrame({'date':candidate_date, 'cal':candidate_cal})
    temp_df = temp_df[temp_df.date==d_target].copy().sort_values('cal', ascending=False)
    calories_target = temp_df.iloc[0].cal
    return solve_for_weeks(model_coefs, weight_init, weight_target, steps_target, calories_target)


In [602]:
foo = solve_for_weeks(model_coefs, 168, 160, 4000, 1625)
foo

[10.17, datetime.date(2020, 7, 7), -0.8, 4000, 1625]

In [612]:
foo = solve_for_calories(model_coefs, 168, 160, 10000, '2020-07-1')
foo

[9.34, datetime.date(2020, 7, 1), -0.8, 10000, 1731]

**TODO**

Solve for calories:
- given a date, calculate weeks to date
- when getting forecast_data, use a small value for dw (e.g. 10 calories)
- instead of interpolating to find answer, use df query approach, e.g. closest weeks that matches target date
- this should give calories and weeks, then calculate lbs/week as above

Gather everything together into custom module:
- function that fits data, returns coefs for linear equation (or model)
- functions that calculate either weeks or calories, fixing all other quantities (returning either weeks, date, lbs_per_week or calories, lbs_per_week

==> this should let you calculate all quantities needed in the table-based dashboard MVP